# IDEA
* Multi-Task RL, e.g., optimize both drawdown and returns
* 

# Reinforcement Learning
## Deep Reinforcement Learning in High Frequency Trading
* https://arxiv.org/pdf/1809.01506.pdf
* Ensemble three one-vs-one MLP, each of which predict binary labels
* Ensemble weight keeps updating through Reinforcement Learning
* 500 trailing tick history and label for 100 forward
* MLP has (10, 10) hidden layers
* Accuracy will be around 70 %
* Only when confidence level is over the threshold, execute trading
* The perticipant percentage is around 10%

## A Deep Reinforcement Learning Framework for the Financial Portfolio Management Problem
* https://arxiv.org/pdf/1706.10059.pdf
* Use Close, High, and Low as input
* Use replay memory and portfolio vector memory for training
* Use 50 trailing history

## Deep Hedging
* https://arxiv.org/pdf/1802.03042.pdf
* Find the hedging strategy through Neural Network
* Optimize convex measure
* Show the experiments over numerically sampled Heston model
* Use (2d, d+15, d+ 15, d) MLP Model, where d is the number of assets

## Agent Inspired Trading Using Recurrent Reinforcement Learning and LSTM Neural Networks
* https://arxiv.org/pdf/1707.07338.pdf
* Direct reinforcement(Not required value function)
* Recurrent Reinforcement Learning using LSTM or RNN
* Optimize Sharp Ratio or Downside Deviation
* Input:
    * Trailing history of price difference
    * Previous position
    * Bias parameter
* Previous position is fed into the input of the output layer
* Validation is done on a single period, which is not trustable


## QLBS: Q-Learner in the Black-Scholes(-Merton) Worlds
* https://arxiv.org/pdf/1712.04609.pdf
* Derive pricing through Q-Learning format